In [ ]:
import numpy as np
import random
import scipy.io
from concurrent.futures import ProcessPoolExecutor
from functools import partial
from cython_decoder import cython_sc_decoding


In [2]:
def int_to_bit(number):
    # Ensure the input number is a NumPy int32
    int32_number = np.array([number], dtype=np.int32)
    # Convert the int32 number to a byte buffer
    byte_representation = int32_number.tobytes()
    # Convert each byte to an 8-bit binary string and concatenate them
    bit_stream = ''.join(format(byte, '08b') for byte in byte_representation)
    # Convert the bit stream to a NumPy array of floats (as in your original bit_array)
    bit_array = np.array([float(bit) for bit in bit_stream], dtype=np.float64)
    return bit_array

def bit_to_int(bit_array):
    # Convert the bit array (floats) to a string of bits
    bit_string = ''.join(str(int(bit)) for bit in bit_array)
    # Convert the bit string to an integer
    int_value = int(bit_string, 2)  # Convert binary string to an integer
    # To get the original int32 value, interpret the bits as an int32 value
    # Handle cases where the original number might be negative by using np.int32
    int32_value = np.frombuffer(int_value.to_bytes(4, byteorder='big'), dtype=np.int32)[0]
    return int32_value

def numpy_to_bit(array):
    tmp_byte=np.frombuffer(array.tobytes(),dtype=np.uint8)
    bit_stream = ''.join(format(byte, '08b') for byte in tmp_byte)
    bit_array = np.array([float(bit) for bit in bit_stream], dtype=np.float64)
    return bit_array

def bit_to_numpy(bit_array):
    bit_stream = ''.join(str(int(bit)) for bit in bit_array)
    byte_array_back = np.array([int(bit_stream[i:i+8], 2) for i in range(0, len(bit_stream), 8)], dtype=np.uint8)
    float_array_back = np.frombuffer(byte_array_back.tobytes(), dtype=np.float32)
    return float_array_back

In [3]:
'''
Encoding and decoding function
'''
def encode(u):
    N = u.shape[0]  # Get the length of u
    n = int(np.log2(N))  # Calculate the log base 2 of N

    if n == 1:
        x = np.array([(u[0] + u[1]) % 2, u[1]])
        return x
    else:
        x1 = encode(np.mod(u[:N//2] + u[N//2:], 2))
        x2 = encode(u[N//2:])
        x = np.concatenate((x1, x2))
        return x

def rvsl(y):
    N = y.shape[0]
    if N == 2:
        return y
    else:
        return np.concatenate((rvsl(y[0:N:2]), rvsl(y[1:N:2])))

def data_generate(bit_array, data_idx):
    u=np.zeros(1024)
    u[data_idx] = bit_array
    x = encode(u)
    x = rvsl(x)
    x = 1-2*x
    return x

def decoding(bit_array, freeze_idx, data_idx):
    # Prepare the necessary arrays and values
    lr0 = np.exp(-(bit_array - 1)**2)
    lr1 = np.exp(-(bit_array + 1)**2)
    lr0_post = lr0 / (lr0 + lr1)
    lr1_post = lr1 / (lr0 + lr1)
    delete_num = 1024 - len(bit_array)
    hd_dec = np.zeros(1024, dtype=np.float64)
    frozen_val = np.zeros(len(freeze_idx), dtype=np.float64)
    pro_prun = np.zeros((1, 2 * 1024 + 1), dtype=np.float64)

    # Call the optimized Cython function
    i_scen_sum, hd_dec_result = cython_sc_decoding(
        lr0_post, lr1_post, freeze_idx.astype(np.float64),
        hd_dec, 1024, 10, 512, frozen_val, delete_num, 0, pro_prun
    )

    # Extract the output for data_idx from hd_dec_result
    data_out = hd_dec_result[data_idx]
    return data_out

In [4]:
N = 1024
n = 10
rate = 0.5
K = round(N*rate)
coding_list = scipy.io.loadmat("1024-3db-d=2-mean.mat")["count_number"]
coding_index = np.argsort(coding_list[:,1])
info_idx = coding_index[:K]
freeze_idx = coding_index[K:]

# sort the final index
info_ni = np.sort(info_idx)
freeze_ni = np.sort(freeze_idx)

In [ ]:
loaded_data = np.load('model.npz')
split_bit = []
codeword_idx = []
bit_array_len = []
codeword_idx.append(0)
total_idx = 0

encode_partial = partial(data_generate, data_idx=info_ni)
for tmp_key in loaded_data.keys():
    tmp_array = loaded_data[tmp_key]
    bit_array = numpy_to_bit(tmp_array)
    bit_array_len.append(len(bit_array))
    for i in range(0,len(bit_array),512):
        sub_array = bit_array[i:i+512]
        # Add 1 at the end of the array
        if len(sub_array) < 512:
            padding = np.ones((512 - len(sub_array)), dtype=bit_array.dtype)
            sub_array = np.concatenate((sub_array, padding))
        split_bit.append(sub_array)
    total_idx += i // 512 + 1
    codeword_idx.append(total_idx)

with ProcessPoolExecutor() as executor:
    codeword_py = list(executor.map(encode_partial, split_bit))

In [8]:
'''
Generate transfer packet get 2^n bits from each codeword.
'''
n = 0
block_len = 2**n
codeword_len = 1024
udp_packet = []
for idx, i in enumerate(range(0, codeword_len, block_len)):
    tmp_packet = []
    packet_idx = int_to_bit(idx)
    tmp_packet = np.concatenate([tmp_codeword[i:i+block_len] for tmp_codeword in codeword_py])
    tmp_udp_packet = np.concatenate((packet_idx,tmp_packet))
    udp_packet.append(tmp_udp_packet)

'''
Data transfer, random drop packets, shuffle the data
'''
loss_ratio = 0.3
num_to_delete = int(len(udp_packet)*loss_ratio)
indices_to_delete = random.sample(range(len(udp_packet)), num_to_delete)
for index in sorted(indices_to_delete, reverse=True):
    del udp_packet[index]
random.shuffle(udp_packet)

# Sorted the shuffle data list
udp_idx=[bit_to_int(tmp_packet[:32].flatten()) for tmp_packet in udp_packet]
sorted_packet_del = np.array([tmp_packet[32:] for _, tmp_packet in sorted(zip(udp_idx,udp_packet))])

sorted_packet = np.zeros((int(codeword_len/block_len), len(codeword_py)*block_len),dtype=sorted_packet_del.dtype)
for i, tmp_idx in enumerate(sorted(udp_idx)):
    sorted_packet[tmp_idx] = sorted_packet_del[i]


restore_codeword = []
for i in range(0,sorted_packet.shape[1],block_len):
    tmp_packet = sorted_packet[:,i:i+block_len].flatten()
    restore_codeword.append(tmp_packet)

# Use decoding algorithm
# decoding_data = []
# for tmp_codeword in restore_codeword:
#     tmp_data = np.array(decoding(tmp_codeword, freeze_ni, info_ni))
#     decoding_data.append(tmp_data)

decode_partial = partial(decoding, freeze_idx=freeze_ni, data_idx=info_ni)

# Run decoding in parallel
with ProcessPoolExecutor() as executor:
    decoding_data = list(executor.map(decode_partial, restore_codeword))

# Use codeword_idx to integrade the decoding_data to bit data, and convert them back to numpy array
restore_array = []
for i, array_len in enumerate(bit_array_len):
    tmp_array = np.concatenate(decoding_data[codeword_idx[i]:codeword_idx[i+1]])
    restore_array.append(tmp_array[:array_len])

receive_array = []
for tmp_bit_array in restore_array:
    receive_array.append(bit_to_numpy(tmp_bit_array.flatten()))


input_array = []
for tmp_key in loaded_data.keys():
    input_array.append(loaded_data[tmp_key].flatten())
for i in range(len(receive_array)):
    if np.all(receive_array[i]==input_array[i]) == False:
        print(i)

In [ ]:
input_array = []
for tmp_key in loaded_data.keys():
    input_array.append(loaded_data[tmp_key].flatten())
for i in range(len(receive_array)):
    if np.all(receive_array[i]==input_array[i]) == False:
        print(i)